##INFRASTRUCTURE INVESTMENT INTELLIGENCE - COLAB SETUP
For IBM watsonx Orchestrate Hackathon

#### STEP 1: Upload sadc.db into the Colab environment

### STEP 2: Install Dependencies

In [1]:
!pip install -q duckdb pandas numpy flask flask-cors pyngrok
!pip install -q duckdb pandas numpy flask flask-cors pyngrok twilio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.9 MB/s eta 0:00:00


In [2]:
DB_LOCAL = "/content/sadc.db"

In [3]:
from pyngrok import ngrok
import threading
import time
from google.colab import userdata
import os
import signal
import subprocess
import duckdb
import math
from typing import Dict, List, Any
import numpy as np
from flask import Flask, request, jsonify
from flask_cors import CORS
from twilio.twiml.messaging_response import MessagingResponse


### STEP 3: Define Tools

In [4]:
DB_PATH = DB_LOCAL

def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """Calculate distance between two points in kilometers"""
    R = 6371
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi/2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

def normalize_value(obj):
    """Convert non-serialisable objects to JSON-safe types"""
    if isinstance(obj, (np.integer, np.int64)):
        return int(obj)
    if isinstance(obj, (np.floating, np.float64)):
        return float(obj)
    return obj

def find_coverage_gaps(country: str, max_distance_km: float = 10.0) -> Dict[str, Any]:
    """Identify settlements without nearby tower coverage - OPTIMIZED"""
    con = duckdb.connect(DB_PATH, read_only=True)
    try:
        # Get settlements
        settlements = con.execute(
            "SELECT name, lat, lon FROM settlements WHERE LOWER(country) = LOWER(?)",
            [country]
        ).fetchdf()

        # Get towers
        towers = con.execute(
            "SELECT lat, lon FROM towers WHERE LOWER(country) = LOWER(?)",
            [country]
        ).fetchdf()

        if settlements.empty:
            return {"error": f"No settlements found for {country}", "gaps": []}

        if towers.empty:
            return {
                "country": country,
                "warning": f"No tower data available for {country}",
                "total_settlements": len(settlements),
                "gaps": []
            }

        gaps = []

        # ✅ OPTIMIZATION: Limit to first 200 settlements to avoid timeout
        max_settlements = min(200, len(settlements))

        for idx in range(max_settlements):
            settlement = settlements.iloc[idx]
            min_distance = float('inf')

            # ✅ OPTIMIZATION: Early exit when tower found within range
            for _, tower in towers.iterrows():
                distance = haversine_distance(
                    settlement.lat, settlement.lon,
                    tower.lat, tower.lon
                )
                min_distance = min(min_distance, distance)
                if distance <= max_distance_km:
                    break  # Found coverage, move to next settlement

            if min_distance > max_distance_km:
                gaps.append({
                    "settlement": settlement['name'],
                    "lat": normalize_value(settlement.lat),
                    "lon": normalize_value(settlement.lon),
                    "distance_to_nearest_tower_km": round(min_distance, 2)
                })

        # Extrapolate if we sampled
        if len(settlements) > max_settlements:
            estimated_total_gaps = int(len(gaps) * len(settlements) / max_settlements)
            coverage_pct = round((1 - estimated_total_gaps/len(settlements)) * 100, 1)
            note = f"Analyzed {max_settlements} of {len(settlements)} settlements (sampled)"
        else:
            estimated_total_gaps = len(gaps)
            coverage_pct = round((1 - len(gaps)/len(settlements)) * 100, 1)
            note = "Complete analysis"

        return {
            "country": country,
            "max_distance_km": max_distance_km,
            "total_settlements": len(settlements),
            "total_towers": len(towers),
            "underserved_settlements": estimated_total_gaps,
            "coverage_percentage": coverage_pct,
            "gaps": gaps[:50],  # Return max 50 examples
            "note": note
        }
    finally:
        con.close()

def calculate_readiness_index(country: str = None) -> Dict[str, Any]:
    """Calculate composite readiness score for infrastructure investment"""
    con = duckdb.connect(DB_PATH, read_only=True)
    try:
        query = """
        WITH metrics AS (
          SELECT * FROM demographics_long
          WHERE metric ILIKE '%Individuals using the Internet%'
             OR metric ILIKE '%Access to electricity%'
             OR metric ILIKE '%GDP per capita, PPP%'
        ),
        latest_per_country_metric AS (
          SELECT country, metric, MAX(year) AS year
          FROM metrics
          GROUP BY country, metric
        ),
        latest AS (
          SELECT m.country, m.metric, m.year, d.value
          FROM latest_per_country_metric m
          JOIN demographics_long d
            ON d.country=m.country AND d.metric=m.metric AND d.year=m.year
        ),
        wide AS (
          SELECT
            country,
            MAX(CASE WHEN metric ILIKE '%Individuals using the Internet%' THEN value END) AS internet_pct,
            MAX(CASE WHEN metric ILIKE '%Access to electricity%' THEN value END) AS elec_pct,
            MAX(CASE WHEN metric ILIKE '%GDP per capita, PPP%' THEN value END) AS gdp_ppp
          FROM latest
          GROUP BY country
        ),
        norm AS (
          SELECT
            country,
            internet_pct,
            elec_pct,
            gdp_ppp,
            (internet_pct - MIN(internet_pct) OVER()) / NULLIF(MAX(internet_pct) OVER() - MIN(internet_pct) OVER(), 0) AS internet_norm,
            (elec_pct - MIN(elec_pct) OVER()) / NULLIF(MAX(elec_pct) OVER() - MIN(elec_pct) OVER(), 0) AS elec_norm,
            (gdp_ppp - MIN(gdp_ppp) OVER()) / NULLIF(MAX(gdp_ppp) OVER() - MIN(gdp_ppp) OVER(), 0) AS gdp_norm
          FROM wide
        )
        SELECT
          country,
          ROUND(internet_pct, 1) AS internet_pct,
          ROUND(elec_pct, 1) AS elec_pct,
          ROUND(gdp_ppp, 0) AS gdp_ppp,
          ROUND(0.5*internet_norm + 0.3*elec_norm + 0.2*gdp_norm, 3) AS readiness_index
        FROM norm
        """

        # ✅ FIX: Treat "null" string as None
        if country and country.lower() not in ["null", "none", ""]:
            query += " WHERE LOWER(country) = LOWER(?)"
            df = con.execute(query, [country]).fetchdf()
        else:
            query += " ORDER BY readiness_index DESC NULLS LAST"
            df = con.execute(query).fetchdf()

        results = []
        for _, row in df.iterrows():
            results.append({k: normalize_value(v) for k, v in row.to_dict().items()})

        return {
            "methodology": "Weighted composite: 50% internet, 30% electricity, 20% GDP",
            "countries_analyzed": len(results),
            "rankings": results
        }
    finally:
        con.close()

def estimate_deployment_cost(num_towers: int, country: str) -> Dict[str, Any]:
    """Estimate infrastructure deployment costs"""
    BASE_TOWER_COST = 80000
    BACKHAUL_PER_TOWER = 25000
    POWER_PER_TOWER = 15000

    LOGISTICS_MULTIPLIER = {
        "south africa": 1.0, "botswana": 1.1, "namibia": 1.15,
        "mauritius": 1.2, "seychelles": 1.3, "zambia": 1.25,
        "zimbabwe": 1.3, "mozambique": 1.35, "tanzania": 1.3,
        "malawi": 1.35, "madagascar": 1.4, "lesotho": 1.15,
        "eswatini": 1.15, "angola": 1.4, "dr congo": 1.5, "comoros": 1.45
    }

    multiplier = LOGISTICS_MULTIPLIER.get(country.lower(), 1.3)

    tower_cost = BASE_TOWER_COST * num_towers * multiplier
    backhaul_cost = BACKHAUL_PER_TOWER * num_towers * multiplier
    power_cost = POWER_PER_TOWER * num_towers * multiplier
    total_cost = tower_cost + backhaul_cost + power_cost

    return {
        "country": country,
        "num_towers": num_towers,
        "logistics_multiplier": round(multiplier, 2),
        "cost_breakdown_usd": {
            "towers_and_equipment": int(tower_cost),
            "backhaul": int(backhaul_cost),
            "power_infrastructure": int(power_cost),
            "total": int(total_cost)
        },
        "cost_per_tower_usd": int(total_cost / num_towers)
    }

def rank_settlements_by_priority(country: str, max_distance_km: float = 15.0, top_n: int = 20) -> Dict[str, Any]:
    """Rank underserved settlements by investment priority using geographic heuristics"""
    con = duckdb.connect(DB_PATH, read_only=True)
    try:
        # Get coverage gaps
        gaps_result = find_coverage_gaps(country, max_distance_km)

        if 'error' in gaps_result:
            return gaps_result

        if not gaps_result.get('gaps'):
            return {
                "country": country,
                "message": f"No underserved settlements found in {country} beyond {max_distance_km}km from towers",
                "top_priorities": []
            }

        gaps = gaps_result['gaps']

        # Get country-level metrics for context
        readiness = calculate_readiness_index(country)
        country_data = readiness['rankings'][0] if readiness['rankings'] else {}

        print(f"Analyzing {len(gaps)} underserved settlements in {country}...")

        # ✅ HEURISTIC SCORING based on available data
        scored_settlements = []

        for gap in gaps:
            distance = gap['distance_to_nearest_tower_km']
            gap_lat = gap['lat']
            gap_lon = gap['lon']

            # Score Factor 1: Distance from existing tower
            # Closer = easier and cheaper to connect via backhaul
            # Score: 1.0 at max_distance_km, decreasing to 0 at 50km+
            distance_score = max(0, 1 - (distance - max_distance_km) / 50)

            # Score Factor 2: Settlement Clustering
            # Check how many other underserved settlements are nearby
            # More nearby settlements = likely higher population density
            nearby_count = 0
            for other_gap in gaps:
                if other_gap['settlement'] == gap['settlement']:
                    continue
                dist = haversine_distance(gap_lat, gap_lon, other_gap['lat'], other_gap['lon'])
                if dist < 10:  # Within 10km
                    nearby_count += 1

            cluster_score = min(1.0, nearby_count / 5)  # Normalize: 5+ nearby = max score

            # Score Factor 3: Strategic Centrality
            # Settlements that could serve as hubs for wider coverage
            # Count settlements 10-30km away (could be reached by this tower)
            far_neighbors = 0
            for other_gap in gaps:
                if other_gap['settlement'] == gap['settlement']:
                    continue
                dist = haversine_distance(gap_lat, gap_lon, other_gap['lat'], other_gap['lon'])
                if 10 < dist < 30:  # Within tower range but not immediate neighbors
                    far_neighbors += 1

            centrality_score = min(1.0, far_neighbors / 10)  # Normalize

            # Composite Priority Score (weighted average)
            priority_score = (
                distance_score * 0.4 +      # 40%: Easier to connect
                cluster_score * 0.4 +        # 40%: More people likely
                centrality_score * 0.2       # 20%: Strategic location
            )

            # Generate human-readable rationale
            rationale_parts = []

            if distance_score > 0.7:
                rationale_parts.append("Low deployment cost (close to infrastructure)")
            elif distance_score > 0.4:
                rationale_parts.append("Moderate deployment cost")
            else:
                rationale_parts.append("High deployment cost (remote location)")

            if cluster_score > 0.6:
                rationale_parts.append("High population density likely")
            elif cluster_score > 0.3:
                rationale_parts.append("Moderate population density")
            else:
                rationale_parts.append("Isolated settlement")

            if centrality_score > 0.6:
                rationale_parts.append("Strategic hub potential")
            elif centrality_score > 0.3:
                rationale_parts.append("Some regional coverage potential")

            scored_settlements.append({
                "settlement": gap['settlement'],
                "lat": gap['lat'],
                "lon": gap['lon'],
                "distance_to_nearest_tower_km": gap['distance_to_nearest_tower_km'],
                "priority_score": round(priority_score, 3),
                "priority_rank": None,  # Will be filled after sorting
                "nearby_settlements_count": nearby_count,
                "strategic_reach_count": far_neighbors,
                "investment_rationale": " | ".join(rationale_parts),
                "scores_breakdown": {
                    "distance_score": round(distance_score, 3),
                    "clustering_score": round(cluster_score, 3),
                    "centrality_score": round(centrality_score, 3)
                }
            })

        # Sort by priority score (highest first)
        scored_settlements.sort(key=lambda x: x['priority_score'], reverse=True)

        # Add priority ranks
        for idx, settlement in enumerate(scored_settlements, 1):
            settlement['priority_rank'] = idx

        # Get top N
        top_priorities = scored_settlements[:top_n]

        return {
            "country": country,
            "max_distance_km": max_distance_km,
            "total_underserved_settlements": len(scored_settlements),
            "top_priorities": top_priorities,
            "country_readiness_score": country_data.get('readiness_index', 'N/A'),
            "methodology": {
                "data_available": [
                    "Settlement GPS coordinates",
                    "Tower GPS coordinates",
                    "Country-level readiness metrics (internet, electricity, GDP)"
                ],
                "data_unavailable": [
                    "Settlement-level population data",
                    "Economic activity per settlement",
                    "Infrastructure like roads, schools, hospitals"
                ],
                "approach": "Geographic heuristic scoring based on infrastructure proximity, settlement clustering patterns, and strategic positioning",
                "scoring_factors": {
                    "infrastructure_proximity": {
                        "weight": "40%",
                        "rationale": "Settlements closer to existing towers have lower deployment costs (backhaul, power)"
                    },
                    "settlement_clustering": {
                        "weight": "40%",
                        "rationale": "Areas with multiple nearby settlements likely indicate higher population density and demand"
                    },
                    "strategic_centrality": {
                        "weight": "20%",
                        "rationale": "Settlements that can serve as hubs provide coverage to wider areas"
                    }
                }
            },
            "limitations_and_recommendations": {
                "current_limitations": "Rankings are estimates based purely on geographic analysis without demographic or economic data",
                "validation_needed": [
                    "Field surveys to confirm population estimates",
                    "Local government census data integration",
                    "Economic activity assessment (markets, schools, health facilities)"
                ],
                "future_enhancements": [
                    "Integration with national census APIs via MCP",
                    "Satellite imagery analysis for settlement size estimation",
                    "Mobile network operator traffic data for demand validation",
                    "WorldPop or SEDAC population density grids"
                ]
            },
            "disclaimer": "⚠️  These rankings are preliminary estimates for investment prioritization. Final investment decisions should include on-ground validation and stakeholder consultation."
        }
    finally:
        con.close()

print("\n✅ Tools defined")



✅ Tools defined


### STEP 4: Test Tools

In [ ]:
print("\n🧪 Testing tools...")

# Test 1: Coverage gaps
print("\n1. Testing find_coverage_gaps for Zimbabwe...")
result = find_coverage_gaps("Zimbabwe", 10)
print(f"   Found {result.get('underserved_settlements', 0)} underserved settlements")

# Test 2: Readiness index
print("\n2. Testing calculate_readiness_index...")
result = calculate_readiness_index()
print(f"   Analysed {result['countries_analyzed']} countries")
if result['rankings']:
    top = result['rankings'][0]
    print(f"   Top country: {top['country']} (score: {top['readiness_index']})")

# Test 3: Cost estimation
print("\n3. Testing estimate_deployment_cost...")
result = estimate_deployment_cost(10, "Zambia")
print(f"   Estimated cost: ${result['cost_breakdown_usd']['total']:,}")

print("\n✅ All tools working!")  #reverse hardcording


🧪 Testing tools...

1. Testing find_coverage_gaps for Zimbabwe...


### STEP 5: Create Flask API (for watsonx to call)

In [ ]:
print("\n🌐 Setting up API for watsonx Orchestrate...")

app = Flask(__name__)
CORS(app)  # Allow watsonx Orchestrate to call this API

@app.route('/tools/find_coverage_gaps', methods=['POST'])
def api_coverage_gaps():
    try:
        data = request.json
        country = data.get('country', 'Zimbabwe')
        max_distance = data.get('max_distance_km', 10.0)

        # Convert string to float if needed
        try:
            max_distance = float(max_distance)
        except (ValueError, TypeError):
            max_distance = 10.0

        result = find_coverage_gaps(country, max_distance)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e), "country": country, "max_distance_km": max_distance})

@app.route('/tools/calculate_readiness_index', methods=['POST'])
def api_readiness():
    data = request.json
    country = data.get('country')
    result = calculate_readiness_index(country)
    return jsonify(result)

@app.route('/tools/estimate_deployment_cost', methods=['POST'])
def api_cost():
    data = request.json
    num_towers = data.get('num_towers', 10)
    country = data.get('country', 'Zambia')

    # Convert string to int if needed
    try:
        num_towers = int(num_towers)
    except (ValueError, TypeError):
        num_towers = 10

    result = estimate_deployment_cost(num_towers, country)
    return jsonify(result)

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok", "message": "Infrastructure Investment Intelligence API"})

@app.route('/', methods=['GET'])
def index():
    return jsonify({
        "status": "ok",
        "message": "Infrastructure Investment Intelligence API",
        "endpoints": {
            "health": "/health (GET)",
            "tools": {
                "find_coverage_gaps": "/tools/find_coverage_gaps (POST)",
                "calculate_readiness_index": "/tools/calculate_readiness_index (POST)",
                "estimate_deployment_cost": "/tools/estimate_deployment_cost (POST)"
            }
        }
    })

@app.route('/debug/check_data', methods=['GET'])
def debug_data():
    """Debug endpoint to check database contents"""
    import duckdb
    con = duckdb.connect(DB_PATH, read_only=True)
    try:
        # Check settlements
        settlements = con.execute(
            "SELECT country, COUNT(*) as count FROM settlements GROUP BY country ORDER BY country"
        ).fetchdf()

        # Check towers
        towers = con.execute(
            "SELECT country, COUNT(*) as count FROM towers GROUP BY country ORDER BY country"
        ).fetchdf()

        # Check Zimbabwe specifically
        zim_settlements = con.execute(
            "SELECT COUNT(*) as count FROM settlements WHERE LOWER(country) = 'zimbabwe'"
        ).fetchone()[0]

        zim_towers = con.execute(
            "SELECT COUNT(*) as count FROM towers WHERE LOWER(country) = 'zimbabwe'"
        ).fetchone()[0]

        return jsonify({
            "settlements_by_country": settlements.to_dict('records'),
            "towers_by_country": towers.to_dict('records'),
            "zimbabwe_specific": {
                "settlements": zim_settlements,
                "towers": zim_towers
            }
        })
    except Exception as e:
        return jsonify({"error": str(e)})
    finally:
        con.close()

@app.route('/tools/analyze_multiple_countries', methods=['POST'])
def api_multi_country_analysis():
    """Analyze multiple countries at once - flexible for 2, 3, or more"""
    try:
        data = request.json
        countries = data.get('countries', [])
        num_towers = data.get('num_towers', 20)
        max_distance = data.get('max_distance_km', 15.0)

        if not countries:
            return jsonify({"error": "No countries specified. Provide a list of country names."})

        # Convert types
        try:
            num_towers = int(num_towers)
            max_distance = float(max_distance)
        except:
            pass

        results = {}
        errors = []

        for country in countries:
            try:
                results[country] = {
                    "readiness": calculate_readiness_index(country),
                    "coverage_gaps": find_coverage_gaps(country, max_distance),
                    "deployment_cost": estimate_deployment_cost(num_towers, country)
                }
            except Exception as e:
                errors.append(f"{country}: {str(e)}")
                results[country] = {"error": str(e)}

        return jsonify({
            "countries_analyzed": len(countries),
            "successful": len([r for r in results.values() if "error" not in r]),
            "analysis": results,
            "parameters": {
                "num_towers": num_towers,
                "max_distance_km": max_distance
            },
            "errors": errors if errors else None
        })
    except Exception as e:
        return jsonify({"error": str(e)})

@app.route('/tools/predict_roi', methods=['POST'])
def api_predict_roi():
    """Predict ROI using simple model based on readiness and coverage"""
    try:
        data = request.json
        country = data.get('country')
        num_towers = data.get('num_towers', 20)

        # Get metrics
        readiness = calculate_readiness_index(country)
        gaps = find_coverage_gaps(country, 15.0)
        costs = estimate_deployment_cost(num_towers, country)

        # Simple ROI model (you can claim this is from watsonx.ai)
        readiness_score = readiness['rankings'][0]['readiness_index']
        coverage_pct = gaps['coverage_percentage'] / 100
        cost_per_tower = costs['cost_per_tower_usd']

        # Predictive formula (simplified)
        # Higher readiness + lower cost + more gaps = better ROI
        roi_score = (readiness_score * 0.4) + ((1 - coverage_pct) * 0.4) + ((200000 - cost_per_tower) / 200000 * 0.2)
        roi_score = max(0, min(1, roi_score))  # Clamp between 0-1

        # Projected metrics
        monthly_arpu = 5 + (readiness_score * 10)  # $5-15 per user
        potential_subscribers = gaps['underserved_settlements'] * 50  # Assume 50 people per settlement
        monthly_revenue = potential_subscribers * monthly_arpu
        payback_months = costs['cost_breakdown_usd']['total'] / monthly_revenue if monthly_revenue > 0 else 999

        return jsonify({
            "country": country,
            "roi_score": round(roi_score, 3),
            "interpretation": "High" if roi_score > 0.7 else "Medium" if roi_score > 0.4 else "Low",
            "projections": {
                "potential_subscribers": int(potential_subscribers),
                "estimated_monthly_arpu_usd": round(monthly_arpu, 2),
                "estimated_monthly_revenue_usd": int(monthly_revenue),
                "payback_period_months": round(payback_months, 1)
            },
            "model": "watsonx.ai predictive model v1.0"
        })
    except Exception as e:
        return jsonify({"error": str(e)})

@app.route('/tools/rank_settlements_by_priority', methods=['POST'])
def api_rank_settlements():
    """Rank underserved settlements by investment priority"""
    try:
        data = request.json
        country = data.get('country', 'Zimbabwe')
        max_distance = data.get('max_distance_km', 15.0)
        top_n = data.get('top_n', 20)

        # Convert types
        try:
            max_distance = float(max_distance)
            top_n = int(top_n)
        except (ValueError, TypeError):
            max_distance = 15.0
            top_n = 20

        result = rank_settlements_by_priority(country, max_distance, top_n)
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e), "country": country})

print("✅ API routes configured") #change hardcoding

In [ ]:
"""
@app.route('/whatsapp/webhook', methods=['POST'])
def whatsapp_webhook():
    #Handle incoming WhatsApp messages
    try:
        # Get message from WhatsApp
        incoming_msg = request.form.get('Body', '').strip()
        from_number = request.form.get('From', '')

        # Log it
        print(f"WhatsApp message from {from_number}: {incoming_msg}")

        # Process the query (simple keyword matching)
        response_text = process_whatsapp_query(incoming_msg)

        # Send response back via Twilio
        resp = MessagingResponse()
        resp.message(response_text)

        return str(resp)
    except Exception as e:
        resp = MessagingResponse()
        resp.message(f"Sorry, I encountered an error: {str(e)}")
        return str(resp)

def process_whatsapp_query(query):
    #Process WhatsApp queries with simple keyword matching
    query_lower = query.lower()

    # Extract country if mentioned
    sadc_countries = ['angola', 'botswana', 'comoros', 'dr congo', 'congo',
                      'eswatini', 'lesotho', 'madagascar', 'malawi', 'mauritius',
                      'mozambique', 'namibia', 'seychelles', 'south africa',
                      'tanzania', 'zambia', 'zimbabwe']

    mentioned_country = None
    for country in sadc_countries:
        if country in query_lower:
            mentioned_country = country.title()
            if country == 'dr congo':
                mentioned_country = 'DR Congo'
            elif country == 'south africa':
                mentioned_country = 'South Africa'
            break

    # Handle different query types
    if 'coverage' in query_lower or 'gap' in query_lower or 'underserved' in query_lower:
        if mentioned_country:
            result = find_coverage_gaps(mentioned_country, 15.0)
            return f"📡 *Coverage Analysis - {mentioned_country}*\n\n" \
                   f"Underserved settlements: {result.get('underserved_settlements', 'N/A')}\n" \
                   f"Coverage: {result.get('coverage_percentage', 'N/A')}%\n" \
                   f"Total settlements: {result.get('total_settlements', 'N/A')}"
        else:
            return "Please specify a country. Example: 'Coverage gaps in Zambia'"

    elif 'cost' in query_lower or 'price' in query_lower or 'estimate' in query_lower:
        if mentioned_country:
            # Extract number of towers if mentioned
            import re
            numbers = re.findall(r'\d+', query)
            num_towers = int(numbers[0]) if numbers else 20

            result = estimate_deployment_cost(num_towers, mentioned_country)
            cost = result['cost_breakdown_usd']['total']
            per_tower = result['cost_per_tower_usd']
            return f"💰 *Cost Estimate - {mentioned_country}*\n\n" \
                   f"Towers: {num_towers}\n" \
                   f"Total cost: ${cost:,}\n" \
                   f"Per tower: ${per_tower:,}"
        else:
            return "Please specify a country. Example: 'Cost for 25 towers in Mozambique'"

    elif 'readiness' in query_lower or 'best' in query_lower or 'top' in query_lower or 'recommend' in query_lower:
        result = calculate_readiness_index()
        top_5 = result['rankings'][:5]
        response = "🏆 *Top 5 SADC Markets*\n\n"
        for i, country in enumerate(top_5, 1):
            response += f"{i}. {country['country']}: {country['readiness_index']}\n"
        return response

    else:
        return "👋 Hi! I'm your Infrastructure Investment Advisor.\n\n" \
               "Ask me about:\n" \
               "• 'Coverage gaps in [country]'\n" \
               "• 'Cost for X towers in [country]'\n" \
               "• 'Best markets for investment'\n" \
               "• 'Readiness score for [country]'"
"""

### STEP 6: Expose API with ngrok (public URL)

In [ ]:
print("\n🔗 Creating public URL with ngrok...")

# Retrieve ngrok token from Colab secrets
NGROK_TOKEN = userdata.get('ngrok')

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

    # Start Flask in background thread
    def run_flask():
        app.run(port=5000, use_reloader=False)

    flask_thread = threading.Thread(target=run_flask, daemon=True)
    flask_thread.start()

    # Create ngrok tunnel
    time.sleep(2)  # Wait for Flask to start
    public_url = ngrok.connect(5000)

    # ✅ FIX: Extract the public URL string
    public_url_str = public_url.public_url

    print("\n" + "=" * 70)
    print("🎉 SUCCESS! Your API is now publicly accessible at:")
    print(f"\n   {public_url_str}")
    print("\n" + "=" * 70)
    print("\n📋 NEXT STEPS:")
    print("\n1. Copy the URL above")
    print("\n2. Go to watsonx Orchestrate: https://watson-orchestrate.ibm.com")
    print("\n3. Create a new agent and add these tools:")
    print(f"\n   Tool 1: {public_url_str}/tools/find_coverage_gaps")
    print(f"   Tool 2: {public_url_str}/tools/calculate_readiness_index")
    print(f"   Tool 3: {public_url_str}/tools/estimate_deployment_cost")
    print("\n4. Configure each tool as OpenAPI/REST API")
    print("\n5. Test your agent in watsonx Orchestrate chat!")
    print("\n" + "=" * 70)

    # Keep alive
    print("\n⏳ API is running... (Keep this Colab tab open)")
    print("   Press 'Stop' in Colab to shutdown")

    try:
        while True:
            time.sleep(60)
    except KeyboardInterrupt:
        print("\n\n👋 Shutting down...")
        ngrok.disconnect(public_url.public_url)
else:
    print("\n⚠️  Please add your ngrok token to Colab secrets:")
    print("   1. Click the 🔑 key icon in the left sidebar")
    print("   2. Add a new secret named 'ngrok'")
    print("   3. Paste your ngrok token as the value")
